In [80]:
%matplotlib inline
import math
import re
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import defaultdict
from scipy.stats import pearsonr
import pandas as pd
from IPython.display import display


In [81]:
DATA_DIR = r"Y:\Studium\3. Sem UniPI\Data Analytics 4 digital Health\Data"

DATASETS = {
    "heart_diagnoses_1": "heart_diagnoses_1.csv",
    "laboratory_events_codes_2": "laboratory_events_codes_2.csv",
    "microbiology_events_codes_3": "microbiology_events_codes_3.csv",
    "procedure_code_4": "procedure_code_4.csv",
}

In [82]:
df = pd.read_csv(f"{DATA_DIR}/{DATASETS['laboratory_events_codes_2']}", index_col=False)

df.columns

Index(['hadm_id', 'charttime', 'value', 'valuenum', 'valueuom',
       'ref_range_lower', 'ref_range_upper', 'flag', 'label', 'fluid',
       'examination_group', 'analysis_batch_id', 'qc_flag', 'ref_range'],
      dtype='object')

# INspections

## A

In [7]:
df.head(10)

,hadm_id,charttime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,label,fluid,examination_group,analysis_batch_id,qc_flag,ref_range
0,29654838.0,2188-01-04 23:43:00,___,198.00,IU/L,29.0,201.00,NaN,Creatine Kinase (CK),Blood,Cardiac Markers,BATCH_218801_N3,OK,NaN
1,29654838.0,2188-01-04 23:43:00,5,5.00,ng/mL,0.0,10.00,NaN,"Creatine Kinase, MB Isoenzyme",Blood,Cardiac Markers,BATCH_218801_C7,OK,NaN
2,29654838.0,2188-01-04 23:43:00,___,0.03,ng/mL,0.0,0.01,abnormal,Troponin T,Blood,Cardiac Markers,BATCH_218801_R4,OK,NaN
3,29654838.0,2188-01-05 06:56:00,1.2,1.20,NaN,0.9,1.10,abnormal,INR(PT),Blood,Coagulation and Hemostasis,BATCH_218801_K7,OK,NaN
4,29654838.0,2188-01-05 06:56:00,12.8,12.80,sec,9.4,12.50,abnormal,PT,Blood,Coagulation and Hemostasis,BATCH_218801_B1,WARN,NaN
5,29654838.0,2188-01-05 06:56:00,85.8,85.80,sec,25.0,36.50,abnormal,PTT,Blood,Coagulation and Hemostasis,BATCH_218801_R1,OK,NaN
6,29654838.0,2188-01-05 06:56:00,___,8.10,%,4.8,5.90,abnormal,% Hemoglobin A1c,Blood,HbA1c,BATCH_218801_O1,OK,Normal range: 70-110
7,29654838.0,2188-01-05 06:56:00,___,186.00,mg/dL,91.0,123.00,abnormal,eAG,Blood,HbA1c,BATCH_218801_P4,OK,NaN
8,29654838.0,2188-01-05 06:56:00,19,19.00,mEq/L,8.0,20.00,NaN,Anion Gap,Blood,Renal Function Tests,BATCH_218801_O5,OK,NaN
9,29654838.0,2188-01-05 06:56:00,20,20.00,mEq/L,22.0,32.00,abnormal,Bicarbonate,Blood,BMP,BATCH_218801_A1,OK,NaN


In [8]:
df.tail(10)

,hadm_id,charttime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,label,fluid,examination_group,analysis_batch_id,qc_flag,ref_range
978493,21557581.0,2145-08-02 08:30:00,29.9,29.90,%,34.0,45.0,abnormal,Hematocrit,Blood,Complete Blood Count (CBC),BATCH_214508_Y8,OK,Normal range: 70-110
978494,21557581.0,2145-08-02 08:30:00,9.9,9.90,g/dL,11.2,15.7,abnormal,Hemoglobin,Blood,Complete Blood Count (CBC),BATCH_214508_D7,OK,NaN
978495,21557581.0,2145-08-02 08:30:00,30.4,30.40,pg,26.0,32.0,NaN,MCH,Blood,Complete Blood Count (CBC),BATCH_214508_J8,WARN,NaN
978496,21557581.0,2145-08-02 08:30:00,33.1,33.10,g/dL,32.0,37.0,NaN,MCHC,Blood,Complete Blood Count (CBC),BATCH_214508_O5,OK,NaN
978497,21557581.0,2145-08-02 08:30:00,92,92.00,fL,82.0,98.0,NaN,MCV,Blood,Complete Blood Count (CBC),BATCH_214508_M3,OK,NaN
978498,21557581.0,2145-08-02 08:30:00,180,180.00,K/uL,150.0,400.0,NaN,Platelet Count,Blood,Complete Blood Count (CBC),BATCH_214508_B1,OK,NaN
978499,21557581.0,2145-08-02 08:30:00,14.5,14.50,%,10.5,15.5,NaN,RDW,Blood,Complete Blood Count (CBC),BATCH_214508_H5,OK,NaN
978500,21557581.0,2145-08-02 08:30:00,3.26,3.26,m/uL,3.9,5.2,abnormal,Red Blood Cells,Blood,Complete Blood Count (CBC),BATCH_214508_N7,OK,NaN
978501,21557581.0,2145-08-02 08:30:00,8.4,8.40,K/uL,4.0,10.0,NaN,White Blood Cells,Blood,Complete Blood Count (CBC),BATCH_214508_Y6,OK,NaN
978502,21557581.0,2145-08-02 08:30:00,47.9,47.90,fL,35.1,46.3,abnormal,RDW-SD,Blood,Complete Blood Count (CBC),BATCH_214508_P9,OK,NaN


## B

In [40]:
df.columns

Index(['hadm_id', 'charttime', 'value', 'valuenum', 'valueuom',
       'ref_range_lower', 'ref_range_upper', 'flag', 'label', 'fluid',
       'examination_group', 'analysis_batch_id', 'qc_flag', 'ref_range'],
      dtype='object')

In [31]:
df[df.columns[:10]].head(10)

,hadm_id,charttime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,label,fluid
0,29654838.0,2188-01-04 23:43:00,NaN,198.00,IU/L,29.0,201.00,NaN,Creatine Kinase (CK),Blood
1,29654838.0,2188-01-04 23:43:00,5.0,5.00,ng/mL,0.0,10.00,NaN,"Creatine Kinase, MB Isoenzyme",Blood
2,29654838.0,2188-01-04 23:43:00,NaN,0.03,ng/mL,0.0,0.01,abnormal,Troponin T,Blood
3,29654838.0,2188-01-05 06:56:00,1.2,1.20,NaN,0.9,1.10,abnormal,INR(PT),Blood
4,29654838.0,2188-01-05 06:56:00,12.8,12.80,sec,9.4,12.50,abnormal,PT,Blood
5,29654838.0,2188-01-05 06:56:00,85.8,85.80,sec,25.0,36.50,abnormal,PTT,Blood
6,29654838.0,2188-01-05 06:56:00,NaN,8.10,%,4.8,5.90,abnormal,% Hemoglobin A1c,Blood
7,29654838.0,2188-01-05 06:56:00,NaN,186.00,mg/dL,91.0,123.00,abnormal,eAG,Blood
8,29654838.0,2188-01-05 06:56:00,19.0,19.00,mEq/L,8.0,20.00,NaN,Anion Gap,Blood
9,29654838.0,2188-01-05 06:56:00,20.0,20.00,mEq/L,22.0,32.00,abnormal,Bicarbonate,Blood


In [11]:
df[df.columns[10:]].head(10)

,examination_group,analysis_batch_id,qc_flag,ref_range
0,Cardiac Markers,BATCH_218801_N3,OK,NaN
1,Cardiac Markers,BATCH_218801_C7,OK,NaN
2,Cardiac Markers,BATCH_218801_R4,OK,NaN
3,Coagulation and Hemostasis,BATCH_218801_K7,OK,NaN
4,Coagulation and Hemostasis,BATCH_218801_B1,WARN,NaN
5,Coagulation and Hemostasis,BATCH_218801_R1,OK,NaN
6,HbA1c,BATCH_218801_O1,OK,Normal range: 70-110
7,HbA1c,BATCH_218801_P4,OK,NaN
8,Renal Function Tests,BATCH_218801_O5,OK,NaN
9,BMP,BATCH_218801_A1,OK,NaN


## C

In [12]:
df.columns

Index(['hadm_id', 'charttime', 'value', 'valuenum', 'valueuom',
       'ref_range_lower', 'ref_range_upper', 'flag', 'label', 'fluid',
       'examination_group', 'analysis_batch_id', 'qc_flag', 'ref_range'],
      dtype='object')

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978503 entries, 0 to 978502
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   hadm_id            978503 non-null  float64
 1   charttime          978503 non-null  object 
 2   value              934025 non-null  object 
 3   valuenum           907317 non-null  float64
 4   valueuom           884585 non-null  object 
 5   ref_range_lower    865544 non-null  float64
 6   ref_range_upper    865544 non-null  float64
 7   flag               343687 non-null  object 
 8   label              978503 non-null  object 
 9   fluid              978503 non-null  object 
 10  examination_group  978503 non-null  object 
 11  analysis_batch_id  978503 non-null  object 
 12  qc_flag            978503 non-null  object 
 13  ref_range          146215 non-null  object 
dtypes: float64(4), object(10)
memory usage: 104.5+ MB


# Data Understanding and Preprocessing, cleaning of DF1 - heart

In [69]:
df.duplicated().sum()

np.int64(2)

In [70]:
#show duplicated rows
df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist()).head(20)

,hadm_id,charttime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,label,fluid,examination_group,analysis_batch_id,qc_flag,ref_range
337781,22665778.0,2114-06-25 17:44:00,NaN,NaN,NaN,NaN,NaN,NaN,Length of Urine Collection,Urine,Urine Test,BATCH_211406_T7,OK,NaN
337786,22665778.0,2114-06-25 17:44:00,NaN,NaN,NaN,NaN,NaN,NaN,Length of Urine Collection,Urine,Urine Test,BATCH_211406_T7,OK,NaN
926329,27638257.0,2189-04-21 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,Length of Urine Collection,Urine,Urine Test,BATCH_218904_R6,OK,NaN
926331,27638257.0,2189-04-21 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,Length of Urine Collection,Urine,Urine Test,BATCH_218904_R6,OK,NaN


In [84]:
df = df.drop_duplicates()

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 978501 entries, 0 to 978502
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   hadm_id            978501 non-null  float64
 1   charttime          978501 non-null  object 
 2   value              934025 non-null  object 
 3   valuenum           907317 non-null  float64
 4   valueuom           884585 non-null  object 
 5   ref_range_lower    865544 non-null  float64
 6   ref_range_upper    865544 non-null  float64
 7   flag               343687 non-null  object 
 8   label              978501 non-null  object 
 9   fluid              978501 non-null  object 
 10  examination_group  978501 non-null  object 
 11  analysis_batch_id  978501 non-null  object 
 12  qc_flag            978501 non-null  object 
 13  ref_range          146215 non-null  object 
dtypes: float64(4), object(10)
memory usage: 112.0+ MB


In [ ]:
for col in df.columns:
    if col == 'charttime' or col == 'hadm_id' or col == 'subject_id':
        continue
    print(f"{col}: {df[col].value_counts()}")

## Check for wrong NaNs / non typical entries in each column

#### Find wrong NaNs

In [ ]:
print("="*80)
print("FIND NON-NUMERICAL ENTRIES IN NUMERICAL COLUMNS")
print("="*80)

# Identify columns that should be numerical
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nColumns already numeric: {len(numeric_cols)}")
print(numeric_cols)

# Check object/string columns that might contain numerical data
object_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nObject/String columns to check: {len(object_cols)}")
print(object_cols)

# For each object column, try to find non-numerical entries
print("\n" + "="*80)
print("CHECKING FOR NON-NUMERICAL ENTRIES")
print("="*80)

non_numerical_summary = {}

for col in object_cols:
    non_numerical_entries = []
    
    for idx, value in df[col].items():
        if pd.isna(value):  # Skip NaN/None
            continue
        
        # Try to convert to float
        try:
            float(value)
        except (ValueError, TypeError):
            non_numerical_entries.append({
                'index': idx,
                'value': value,
                'type': type(value).__name__
            })
    
    # Store summary
    if non_numerical_entries:
        non_numerical_summary[col] = non_numerical_entries
        
        print(f"\n{'─'*80}")
        print(f"Column: '{col}' | Non-numerical entries: {len(non_numerical_entries)}")
        print(f"{'─'*80}")
        
        # Get unique non-numerical values
        unique_values = list(set([e['value'] for e in non_numerical_entries]))
        print(f"Unique non-numerical values ({len(unique_values)}):")
        for val in sorted(unique_values)[:20]:  # Show first 20
            count = sum(1 for e in non_numerical_entries if e['value'] == val)
            print(f"  • '{val}' — appears {count} times")
        
        if len(unique_values) > 20:
            print(f"  ... and {len(unique_values) - 20} more")
        
        # Show sample rows
        print(f"\nSample rows with non-numerical entries:")
        for entry in non_numerical_entries[:5]:
            print(f"  Index {entry['index']}: {entry['value']!r} ({entry['type']})")

# Summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nColumns with non-numerical entries: {len(non_numerical_summary)}")

for col, entries in non_numerical_summary.items():
    total_rows = len(df)
    non_num_count = len(entries)
    pct = (non_num_count / total_rows) * 100
    unique_count = len(set([e['value'] for e in entries]))
    
    print(f"\n{col}:")
    print(f"  Non-numerical rows: {non_num_count} ({pct:.2f}%)")
    print(f"  Unique non-numerical values: {unique_count}")
    print(f"  Numerical rows: {total_rows - non_num_count}")

# Optional: Create a detailed report
if non_numerical_summary:
    print("\n" + "="*80)
    print("DETAILED REPORT - ALL NON-NUMERICAL ENTRIES")
    print("="*80)
    
    for col in non_numerical_summary:
        print(f"\n{col}:")
        entries_df = pd.DataFrame(non_numerical_summary[col])
        # Count occurrences
        value_counts = entries_df['value'].value_counts()
        print(value_counts)

- valueuom: has '' , remove; Pos/Neg == +/-; U ??
- value has wrong entries inspect and extrat if possible to valuenum


### Handle Value wrong nans, then extract missing from value into new column valuenum_merged if possible

In [85]:
# chec k only value column: show all unique non-numerical entries in 'value' column
non_numerical_values = []
for idx, value in df['value'].items():
    if pd.isna(value):  # Skip NaN/None
        continue
    
    # Try to convert to float
    try:
        float(value)
    except (ValueError, TypeError):
        non_numerical_values.append({
            'index': idx,
            'value': value,
            'type': type(value).__name__
        })
print(f"\n{'─'*80}")
print(f"Column: 'value' | Non-numerical entries: {len(non_numerical_values)}")
print(f"{'─'*80}")
# Get unique non-numerical values
unique_values = list(set([e['value'] for e in non_numerical_values]))
print(f"Unique non-numerical values ({len(unique_values)}):")
for val in sorted(unique_values):
    count = sum(1 for e in non_numerical_values if e['value'] == val)
    print(f"  • '{val}' — appears {count} times")   
    


────────────────────────────────────────────────────────────────────────────────
Column: 'value' | Non-numerical entries: 94777
────────────────────────────────────────────────────────────────────────────────
Unique non-numerical values (222):
  • '/10.' — appears 6 times
  • '/11.' — appears 3 times
  • '/12.' — appears 17 times
  • '/13.' — appears 6 times
  • '/14.' — appears 14 times
  • '/15.' — appears 9 times
  • '/16.' — appears 30 times
  • '/17.' — appears 13 times
  • '/18.' — appears 29 times
  • '/19.' — appears 7 times
  • '/20.' — appears 23 times
  • '/21.' — appears 10 times
  • '/22.' — appears 13 times
  • '/23.' — appears 6 times
  • '/24.' — appears 21 times
  • '/25.' — appears 18 times
  • '/26.' — appears 6 times
  • '/27.' — appears 8 times
  • '/28.' — appears 5 times
  • '/29.' — appears 1 times
  • '/30.' — appears 9 times
  • '/31.' — appears 5 times
  • '/32.' — appears 6 times
  • '/33.' — appears 2 times
  • '/34.' — appears 1 times
  • '/35.' — appears

FINDINGS

=> in col value, We want to convert '___' and 'NONE' 'ERROR' to np.nan!

=> then we create a new col value_extracted (float64) out of col value where:
- we can calculate as float complete / like 20/0 but only if there is anumber before and after the /! => complete
- we can take the middle point of complete ranges like '80-160'
- we can calculate a float value of comparisons with < > by sub/add 0.1 to the number, eg. '>1.050' => 1.150 or '<1' => 0.9
- the rest is to nuemic error coerce put to NaN.

=> then, we fill np.nan entries in col valuenum with values from valUe_extracted if they are not nan and tell me the amount of filled rows and show examples beffore and after

In [93]:
#first sanity check if needed
# check are there rows where valuenum has nan but value has a value?a
len(df[(df['valuenum'].isna()) & (df['value'].notna())])

24056

In [87]:
print("="*80)
print("STEP 1: CONVERT PLACEHOLDER STRINGS TO NaN")
print("="*80)

# Define placeholder patterns
placeholders = ['___', 'NONE', 'ERROR']

print(f"\nPlaceholders to convert: {placeholders}")
print(f"Before: {df['value'].isna().sum()} NaN values")

# Convert placeholders to NaN (case-insensitive)
for placeholder in placeholders:
    mask = df['value'].astype(str).str.lower() == placeholder.lower()
    count = mask.sum()
    df.loc[mask, 'value'] = np.nan
    print(f"  Converted '{placeholder}': {count} rows")

print(f"After: {df['value'].isna().sum()} NaN values")

STEP 1: CONVERT PLACEHOLDER STRINGS TO NaN

Placeholders to convert: ['___', 'NONE', 'ERROR']
Before: 44476 NaN values
  Converted '___': 68645 rows
  Converted 'NONE': 2070 rows
  Converted 'ERROR': 6 rows
After: 115197 NaN values


In [88]:
print("="*80)
print("STEP 2: EXTRACT NUMERIC VALUES FROM 'value' COLUMN")
print("="*80)

def extract_numeric_from_value(x):
    """
    Extract numeric values from various string formats:
    - Simple numbers: '123' => 123
    - Decimals: '123.45' => 123.45
    - Divisions: '20/10' => 2.0
    - Ranges: '80-160' => 120 (midpoint)
    - Comparisons: '>1.050' => 1.150, '<1' => 0.9
    """
    
    if pd.isna(x):
        return np.nan
    
    x_str = str(x).strip()
    
    # Try direct float conversion
    try:
        return float(x_str)
    except ValueError:
        pass
    
    # Handle divisions (e.g., '20/10')
    if '/' in x_str:
        parts = x_str.split('/')
        try:
            if len(parts) == 2 and parts[0].strip() and parts[1].strip():
                num1 = float(parts[0].strip())
                num2 = float(parts[1].strip())
                if num2 != 0:  # Avoid division by zero
                    return num1 / num2
        except (ValueError, ZeroDivisionError):
            pass
    
    # Handle ranges (e.g., '80-160')
    if '-' in x_str and not x_str.startswith('-'):
        parts = x_str.split('-')
        try:
            if len(parts) == 2 and parts[0].strip() and parts[1].strip():
                num1 = float(parts[0].strip())
                num2 = float(parts[1].strip())
                return (num1 + num2) / 2  # Midpoint
        except ValueError:
            pass
    
    # Handle comparisons (e.g., '>1.050' => 1.150, '<1' => 0.9)
    comparison_match = re.match(r'^([<>]=?)(\d*\.?\d+)$', x_str.strip())
    if comparison_match:
        operator = comparison_match.group(1)
        try:
            num = float(comparison_match.group(2))
            if operator == '>':
                return num + 0.1
            elif operator == '>=':
                return num
            elif operator == '<':
                return num - 0.1
            elif operator == '<=':
                return num
        except ValueError:
            pass
    
    # If nothing worked, return NaN
    return np.nan

# Apply extraction
df['value_extracted'] = df['value'].apply(extract_numeric_from_value)

print(f"\nExtraction complete!")
print(f"Non-null values extracted: {df['value_extracted'].notna().sum():,}")
print(f"Failed extractions (NaN): {df['value_extracted'].isna().sum():,}")

STEP 2: EXTRACT NUMERIC VALUES FROM 'value' COLUMN

Extraction complete!
Non-null values extracted: 840,559
Failed extractions (NaN): 137,942


In [90]:
print("="*80)
print("STEP 3: SHOW EXAMPLES OF EXTRACTED VALUES (Complex Formats Only)")
print("="*80)

# Find rows with successful extractions
extracted_rows = df[df['value_extracted'].notna() & df['value'].notna()].copy()

print(f"\nTotal rows with extracted values: {len(extracted_rows):,}\n")

# ────────────────────────────────────────────────────────────────────────────
# COMPARISONS: >, <, >=, <=
# ────────────────────────────────────────────────────────────────────────────
print("="*80)
print("EXAMPLES: COMPARISONS (>, <, >=, <=)")
print("="*80)

comparison_pattern = r'^([<>]=?)(\d*\.?\d+)$'
comparison_examples = []

for idx, row in extracted_rows.iterrows():
    original = str(row['value']).strip()
    if re.match(comparison_pattern, original):
        comparison_examples.append({
            'original': original,
            'extracted': row['value_extracted']
        })

if comparison_examples:
    # Show first 15 unique examples
    unique_comparisons = []
    seen = set()
    for ex in comparison_examples:
        if ex['original'] not in seen:
            unique_comparisons.append(ex)
            seen.add(ex['original'])
            if len(unique_comparisons) >= 15:
                break
    
    print(f"\nFound {len(comparison_examples)} comparison values\n")
    print(f"{'Original':<20} {'Extracted':<15} {'Logic':<40}")
    print("─" * 75)
    
    for ex in unique_comparisons:
        original = ex['original']
        extracted = ex['extracted']
        
        # Explain the logic
        if original.startswith('>'):
            if original.startswith('>='):
                logic = f"'{original}' → value as-is"
            else:
                num = float(original[1:])
                logic = f"'{original}' → {num} + 0.1 = {extracted}"
        elif original.startswith('<'):
            if original.startswith('<='):
                logic = f"'{original}' → value as-is"
            else:
                num = float(original[1:])
                logic = f"'{original}' → {num} - 0.1 = {extracted}"
        
        print(f"{original:<20} {extracted:<15.3f} {logic:<40}")
else:
    print("\nNo comparison values found")

# ────────────────────────────────────────────────────────────────────────────
# RANGES: 80-160, 0-2, etc.
# ────────────────────────────────────────────────────────────────────────────
print("\n" + "="*80)
print("EXAMPLES: RANGES (e.g., 80-160, 0-2)")
print("="*80)

range_pattern = r'^(\d+\.?\d*)-(\d+\.?\d*)$'
range_examples = []

for idx, row in extracted_rows.iterrows():
    original = str(row['value']).strip()
    if re.match(range_pattern, original) and not original.startswith('-'):
        range_examples.append({
            'original': original,
            'extracted': row['value_extracted']
        })

if range_examples:
    # Show first 15 unique examples
    unique_ranges = []
    seen = set()
    for ex in range_examples:
        if ex['original'] not in seen:
            unique_ranges.append(ex)
            seen.add(ex['original'])
            if len(unique_ranges) >= 15:
                break
    
    print(f"\nFound {len(range_examples)} range values\n")
    print(f"{'Original':<20} {'Extracted':<15} {'Logic (Midpoint)':<40}")
    print("─" * 75)
    
    for ex in unique_ranges:
        original = ex['original']
        extracted = ex['extracted']
        
        # Calculate and show logic
        parts = original.split('-')
        num1 = float(parts[0])
        num2 = float(parts[1])
        midpoint = (num1 + num2) / 2
        
        logic = f"({num1} + {num2}) / 2 = {midpoint}"
        
        print(f"{original:<20} {extracted:<15.3f} {logic:<40}")
else:
    print("\nNo range values found")

# ────────────────────────────────────────────────────────────────────────────
# DIVISIONS: 20/10, 15/3, etc.
# ────────────────────────────────────────────────────────────────────────────
print("\n" + "="*80)
print("EXAMPLES: DIVISIONS (e.g., 20/10, 15/3)")
print("="*80)

division_pattern = r'^(\d+\.?\d*)/(\d+\.?\d*)$'
division_examples = []

for idx, row in extracted_rows.iterrows():
    original = str(row['value']).strip()
    if re.match(division_pattern, original):
        division_examples.append({
            'original': original,
            'extracted': row['value_extracted']
        })

if division_examples:
    # Show first 15 unique examples
    unique_divisions = []
    seen = set()
    for ex in division_examples:
        if ex['original'] not in seen:
            unique_divisions.append(ex)
            seen.add(ex['original'])
            if len(unique_divisions) >= 15:
                break
    
    print(f"\nFound {len(division_examples)} division values\n")
    print(f"{'Original':<20} {'Extracted':<15} {'Logic':<40}")
    print("─" * 75)
    
    for ex in unique_divisions:
        original = ex['original']
        extracted = ex['extracted']
        
        # Calculate and show logic
        parts = original.split('/')
        num1 = float(parts[0])
        num2 = float(parts[1])
        result = num1 / num2 if num2 != 0 else float('nan')
        
        logic = f"{num1} / {num2} = {result}"
        
        print(f"{original:<20} {extracted:<15.3f} {logic:<40}")
else:
    print("\nNo division values found")

# ────────────────────────────────────────────────────────────────────────────
# SUMMARY STATISTICS
# ────────────────────────────────────────────────────────────────────────────
print("\n" + "="*80)
print("EXTRACTION SUMMARY")
print("="*80)

print(f"\nTotal extracted values: {len(extracted_rows):,}")
print(f"  • Comparisons:  {len(comparison_examples):,} ({len(comparison_examples)/len(extracted_rows)*100:.1f}%)")
print(f"  • Ranges:       {len(range_examples):,} ({len(range_examples)/len(extracted_rows)*100:.1f}%)")
print(f"  • Divisions:    {len(division_examples):,} ({len(division_examples)/len(extracted_rows)*100:.1f}%)")
print(f"  • Other:        {len(extracted_rows) - len(comparison_examples) - len(range_examples) - len(division_examples):,}")

print(f"\n{'─'*80}")
print("STATISTICS OF EXTRACTED VALUES")
print(f"{'─'*80}")
print(df['value_extracted'].describe())

STEP 3: SHOW EXAMPLES OF EXTRACTED VALUES (Complex Formats Only)

Total rows with extracted values: 840,559

EXAMPLES: COMPARISONS (>, <, >=, <=)

Found 177 comparison values

Original             Extracted       Logic                                   
───────────────────────────────────────────────────────────────────────────
<1                   0.900           '<1' → 1.0 - 0.1 = 0.9                  
>=1.035              1.035           '>=1.035' → value as-is                 
>50                  50.100          '>50' → 50.0 + 0.1 = 50.1               
>300                 300.100         '>300' → 300.0 + 0.1 = 300.1            
>1.035               1.135           '>1.035' → 1.035 + 0.1 = 1.135          
>1.030               1.130           '>1.030' → 1.03 + 0.1 = 1.1300000000000001
>1000                1000.100        '>1000' → 1000.0 + 0.1 = 1000.1         
>1.050               1.150           '>1.050' → 1.05 + 0.1 = 1.1500000000000001

EXAMPLES: RANGES (e.g., 80-160, 0-2)

Fou

In [91]:
print("="*80)
print("STEP 4: MERGE valuenum + value_extracted")
print("="*80)

# Store counts before merge
valuenum_before = df['valuenum'].notna().sum()
value_extracted_only = (df['valuenum'].isna() & df['value_extracted'].notna()).sum()

print(f"\nBefore merge:")
print(f"  valuenum (non-null):           {valuenum_before:,}")
print(f"  value_extracted (non-null):    {df['value_extracted'].notna().sum():,}")
print(f"  Can be filled from extraction:  {value_extracted_only:,}")

# Merge: prefer valuenum, fallback to value_extracted
df['valuenum_merged'] = df['valuenum'].fillna(df['value_extracted'])

valuenum_after = df['valuenum_merged'].notna().sum()
newly_filled = valuenum_after - valuenum_before

print(f"\nAfter merge:")
print(f"  valuenum_merged (non-null):   {valuenum_after:,}")
print(f"  Newly filled from extraction:   {newly_filled:,} (+{(newly_filled/len(df)*100):.2f}%)")
print(f"  Total improvement:              {valuenum_after - df['valuenum'].isna().sum():,} rows")

print(f"\nData type: {df['valuenum_merged'].dtype}")
print(f"\nStatistics:")
print(df['valuenum_merged'].describe())

STEP 4: MERGE valuenum + value_extracted

Before merge:
  valuenum (non-null):           907,317
  value_extracted (non-null):    840,559
  Can be filled from extraction:  1,311

After merge:
  valuenum_merged (non-null):   908,628
  Newly filled from extraction:   1,311 (+0.13%)
  Total improvement:              837,444 rows

Data type: float64

Statistics:
count    908628.000000
mean         67.258029
std        2174.021247
min        -743.000000
25%           4.300000
50%          17.000000
75%          60.000000
max      886449.000000
Name: valuenum_merged, dtype: float64


In [92]:
print("="*80)
print("STEP 5: SHOW BEFORE/AFTER EXAMPLES")
print("="*80)

# Find rows that were newly filled
newly_filled_rows = df[(df['valuenum'].isna()) & (df['value_extracted'].notna())].copy()

print(f"\nTotal newly filled rows: {len(newly_filled_rows):,}\n")
print("BEFORE → AFTER Examples:\n")
print(f"{'Original value':<20} {'Extracted':<15} {'valuenum (before)':<20} {'valuenum_merged (after)':<20}")
print("─" * 75)

for i, (idx, row) in enumerate(newly_filled_rows.head(20).iterrows()):
    original = str(row['value'])[:19]
    extracted = f"{row['value_extracted']:.3f}" if pd.notna(row['value_extracted']) else "NaN"
    before = "NaN"
    after = f"{row['valuenum_merged']:.3f}"
    
    print(f"{original:<20} {extracted:<15} {before:<20} {after:<20}")

print(f"\n{'─'*75}")
print(f"\nCoverage improvement:")
print(f"  Before: {(df['valuenum'].notna().sum() / len(df) * 100):.2f}% coverage")
print(f"  After:  {(df['valuenum_merged'].notna().sum() / len(df) * 100):.2f}% coverage")
print(f"  Gain:   {(newly_filled / len(df) * 100):.2f}%")

STEP 5: SHOW BEFORE/AFTER EXAMPLES

Total newly filled rows: 1,311

BEFORE → AFTER Examples:

Original value       Extracted       valuenum (before)    valuenum_merged (after)
───────────────────────────────────────────────────────────────────────────
0-2                  1.000           NaN                  1.000               
0-2                  1.000           NaN                  1.000               
0-2                  1.000           NaN                  1.000               
6-10                 8.000           NaN                  8.000               
0-2                  1.000           NaN                  1.000               
11-20                15.500          NaN                  15.500              
0-2                  1.000           NaN                  1.000               
0-2                  1.000           NaN                  1.000               
3-5                  4.000           NaN                  4.000               
0-2                  1.000           

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 978501 entries, 0 to 978502
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   hadm_id            978501 non-null  float64
 1   charttime          978501 non-null  object 
 2   value              863304 non-null  object 
 3   valuenum           907317 non-null  float64
 4   valueuom           884585 non-null  object 
 5   ref_range_lower    865544 non-null  float64
 6   ref_range_upper    865544 non-null  float64
 7   flag               343687 non-null  object 
 8   label              978501 non-null  object 
 9   fluid              978501 non-null  object 
 10  examination_group  978501 non-null  object 
 11  analysis_batch_id  978501 non-null  object 
 12  qc_flag            978501 non-null  object 
 13  ref_range          146215 non-null  object 
 14  value_extracted    840559 non-null  float64
 15  valuenum_merged    908628 non-null  float64
dtypes: floa

 ## Convert datetimes

In [95]:
cols = ['charttime',]  

for col in cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"{col}: parsed {df[col].notna().sum()} values, {df[col].isna().sum()} NaT")

display(df[[c for c in df.columns if c in cols]])

charttime: parsed 978501 values, 0 NaT


,charttime
0,2188-01-04 23:43:00
1,2188-01-04 23:43:00
2,2188-01-04 23:43:00
3,2188-01-05 06:56:00
4,2188-01-05 06:56:00
...,...
978498,2145-08-02 08:30:00
978499,2145-08-02 08:30:00
978500,2145-08-02 08:30:00
978501,2145-08-02 08:30:00


## Little intermed inspection 

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978503 entries, 0 to 978502
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   hadm_id            978503 non-null  float64       
 1   charttime          978503 non-null  datetime64[ns]
 2   value              839248 non-null  object        
 3   valuenum           907317 non-null  float64       
 4   valueuom           884585 non-null  object        
 5   ref_range_lower    865544 non-null  float64       
 6   ref_range_upper    865544 non-null  float64       
 7   flag               343687 non-null  object        
 8   label              978503 non-null  object        
 9   fluid              978503 non-null  object        
 10  examination_group  978503 non-null  object        
 11  analysis_batch_id  978503 non-null  object        
 12  qc_flag            978503 non-null  object        
 13  ref_range          146215 non-null  object  

In [35]:
df[df.columns[:10]].head(10)

,hadm_id,charttime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,label,fluid
0,29654838.0,2188-01-04 23:43:00,NaN,198.00,IU/L,29.0,201.00,NaN,Creatine Kinase (CK),Blood
1,29654838.0,2188-01-04 23:43:00,5.0,5.00,ng/mL,0.0,10.00,NaN,"Creatine Kinase, MB Isoenzyme",Blood
2,29654838.0,2188-01-04 23:43:00,NaN,0.03,ng/mL,0.0,0.01,abnormal,Troponin T,Blood
3,29654838.0,2188-01-05 06:56:00,1.2,1.20,NaN,0.9,1.10,abnormal,INR(PT),Blood
4,29654838.0,2188-01-05 06:56:00,12.8,12.80,sec,9.4,12.50,abnormal,PT,Blood
5,29654838.0,2188-01-05 06:56:00,85.8,85.80,sec,25.0,36.50,abnormal,PTT,Blood
6,29654838.0,2188-01-05 06:56:00,NaN,8.10,%,4.8,5.90,abnormal,% Hemoglobin A1c,Blood
7,29654838.0,2188-01-05 06:56:00,NaN,186.00,mg/dL,91.0,123.00,abnormal,eAG,Blood
8,29654838.0,2188-01-05 06:56:00,19.0,19.00,mEq/L,8.0,20.00,NaN,Anion Gap,Blood
9,29654838.0,2188-01-05 06:56:00,20.0,20.00,mEq/L,22.0,32.00,abnormal,Bicarbonate,Blood


In [ ]:
df[df.columns[10:]].head(10)

## Handle missing values

In [36]:
# print sum of all missing values per column
for col in df.columns:
    missing_count = df[col].isna().sum()
    if missing_count > 0:
        print(f"Column '{col}': {missing_count} missing values")

Column 'value': 139255 missing values
Column 'valuenum': 71186 missing values
Column 'valueuom': 93918 missing values
Column 'ref_range_lower': 112959 missing values
Column 'ref_range_upper': 112959 missing values
Column 'flag': 634816 missing values
Column 'ref_range': 832288 missing values
Column 'value_extracted': 139255 missing values
Column 'value_numeric': 71186 missing values


- no dod entry means not dead?  => convert to has_died variable
- for xray, cath, ecg, ... => combine to one/two variable anyways (0/1 or 0/6), missing means this test was not performed
. chief complaint and invasions are text columns => dismiss anyways
- anchor_year: ?

## Do scatterpltos / distr / etc to check for dataqualtiy

### Do correlation matrix

## Non numerical correlation

## Save

In [ ]:
df.to_csv(f"{DATA_DIR}/heart_diagnoses_1_cleaned.csv", index=False)

In [ ]:
# load already cleaned to skip first steps
#df = pd.read_csv(f"{DATA_DIR}/heart_diagnoses_1_cleaned.csv")

In [ ]:
df.info()

## add subject id from df1, df3, and df4

## Create features and slim version

In [ ]:
df_slim = df.copy()

In [ ]:
df_slim.columns

In [ ]:
df_slim[df_slim.columns[:10]]

In [ ]:
df_slim[df_slim.columns[10:]]

### Create features
- var for exams
- var for charttime?
- var for dod (is_dead?), anchor_year?

In [ ]:
# create is_dead variable from dod
df_slim['is_dead'] = df_slim['dod'].notna().astype(int)

In [ ]:
# create: A total exam count variable n_imaging_tests out of 'X-ray', 'CT', 'Ultrasound', 'CATH', 'ECG', 'MRI' (text columns, np.nan means no exam done)
# Define exam columns
exam_cols = ['X-ray', 'CT', 'Ultrasound', 'CATH', 'ECG', 'MRI']
exam_cols = [col for col in exam_cols if col in df_slim.columns]

# 1. TOTAL EXAM COUNT: n_imaging_tests
print("\n" + "="*80)
print("FEATURE 1: Total Exam Count (n_imaging_tests)")
print("="*80)

df_slim['n_imaging_tests'] = df_slim[exam_cols].notna().sum(axis=1)

print(f"\nDistribution of total exams:")
print(df_slim['n_imaging_tests'].value_counts().sort_index())
print(f"\nStatistics:")
print(df_slim['n_imaging_tests'].describe())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
ax1.hist(df_slim['n_imaging_tests'], bins=range(0, 8), edgecolor='black', alpha=0.7, color='steelblue')
ax1.set_xlabel('Number of Imaging Tests', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax1.set_title('Distribution: Total Number of Imaging Tests', fontsize=13, fontweight='bold')
ax1.grid(alpha=0.3, axis='y')

# Bar plot with percentages
ax2 = axes[1]
counts = df_slim['n_imaging_tests'].value_counts().sort_index()
percentages = (counts / len(df_slim) * 100).round(2)
bars = ax2.bar(counts.index, counts.values, color='steelblue', edgecolor='black', alpha=0.7)

# Add percentage labels on bars
for i, (idx, count) in enumerate(counts.items()):
    pct = percentages[idx]
    ax2.text(idx, count + 5, f'{pct}%', ha='center', fontsize=10, fontweight='bold')

ax2.set_xlabel('Number of Imaging Tests', fontsize=12, fontweight='bold')
ax2.set_ylabel('Count', fontsize=12, fontweight='bold')
ax2.set_title('Count & Percentage: Imaging Tests per Patient', fontsize=13, fontweight='bold')
ax2.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(r'../plots/1.1_df1_imaging_tests_count.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# create: (has_radiology_exam, has_cardiac_exam): Radiology = X-ray + CT + MRI + Ultrasound Cardiac diagnostics = ECG + CATH variable out of 'X-ray', 'CT', 'Ultrasound', 'CATH', 'ECG', 'MRI' (text columns, np.nan means no exam done)

# 2. CATEGORICAL FEATURES: has_radiology_exam, has_cardiac_exam
# Radiology exams: X-ray, CT, MRI, Ultrasound
radiology_cols = ['X-ray', 'CT', 'MRI', 'Ultrasound']
radiology_cols = [col for col in radiology_cols if col in df_slim.columns]

# Cardiac exams: ECG, CATH
cardiac_cols = ['ECG', 'CATH']
cardiac_cols = [col for col in cardiac_cols if col in df_slim.columns]

df_slim['has_radiology_exam'] = df_slim[radiology_cols].notna().any(axis=1).astype(int)
df_slim['has_cardiac_exam'] = df_slim[cardiac_cols].notna().any(axis=1).astype(int)

# Combine for exam_type: 0=none, 1=radiology only, 2=cardiac only, 3=both
df_slim['exam_type'] = (
    (df_slim['has_radiology_exam'].astype(int) * 1) + 
    (df_slim['has_cardiac_exam'].astype(int) * 2)
)

exam_type_map = {
    0: 'No Exams',
    1: 'Radiology Only',
    2: 'Cardiac Only',
    3: 'Both Radiology & Cardiac'
}
df_slim['exam_type_label'] = df_slim['exam_type'].map(exam_type_map)

print(f"\nRadiology columns: {radiology_cols}")
print(f"Cardiac columns: {cardiac_cols}\n")

print("has_radiology_exam distribution:")
print(df_slim['has_radiology_exam'].value_counts())
print(f"  → {(df_slim['has_radiology_exam'] == 1).sum()} patients ({(df_slim['has_radiology_exam'].mean()*100):.1f}%) had radiology exams\n")

print("has_cardiac_exam distribution:")
print(df_slim['has_cardiac_exam'].value_counts())
print(f"  → {(df_slim['has_cardiac_exam'] == 1).sum()} patients ({(df_slim['has_cardiac_exam'].mean()*100):.1f}%) had cardiac exams\n")

print("exam_type (combined) distribution:")
print(df_slim['exam_type_label'].value_counts())
print(f"\nBreakdown:")
for exam_type in sorted(df_slim['exam_type'].unique()):
    count = (df_slim['exam_type'] == exam_type).sum()
    pct = (count / len(df_slim)) * 100
    print(f"  {exam_type_map[exam_type]:25s}: {count:5d} ({pct:5.1f}%)")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# # Plot 1: Radiology exam distribution
# ax1 = axes[0, 0]
# radiology_counts = df_slim['has_radiology_exam'].value_counts().sort_index()
# colors_rad = ['#ff6b6b', '#51cf66']
# # Use .iloc for positional indexing, or access by value: radiology_counts[0], radiology_counts[1]
# ax1.bar(['No Radiology', 'Has Radiology'], 
#         [radiology_counts.iloc[0], radiology_counts.iloc[1]], 
#         color=colors_rad, edgecolor='black', alpha=0.7, linewidth=2)
# for i, v in enumerate([radiology_counts.iloc[0], radiology_counts.iloc[1]]):
#     pct = (v / len(df_slim)) * 100
#     ax1.text(i, v + 20, f'{v}\n({pct:.1f}%)', ha='center', fontsize=11, fontweight='bold')
# ax1.set_ylabel('Count', fontsize=12, fontweight='bold')
# ax1.set_title('Radiology Exams: X-ray, CT, MRI, Ultrasound', fontsize=13, fontweight='bold')
# ax1.grid(alpha=0.3, axis='y')

# Plot 2: Cardiac exam distribution
ax2 = axes[0, 1]
cardiac_counts = df_slim['has_cardiac_exam'].value_counts().sort_index()
colors_card = ['#ff6b6b', '#4c6ef5']
ax2.bar(['No Cardiac', 'Has Cardiac'], 
        [cardiac_counts.iloc[0], cardiac_counts.iloc[1]], 
        color=colors_card, edgecolor='black', alpha=0.7, linewidth=2)
for i, v in enumerate([cardiac_counts.iloc[0], cardiac_counts.iloc[1]]):
    pct = (v / len(df_slim)) * 100
    ax2.text(i, v + 20, f'{v}\n({pct:.1f}%)', ha='center', fontsize=11, fontweight='bold')
ax2.set_ylabel('Count', fontsize=12, fontweight='bold')
ax2.set_title('Cardiac Exams: ECG, CATH', fontsize=13, fontweight='bold')
ax2.grid(alpha=0.3, axis='y')

# ...existing code...

# Plot 3: Combined exam type distribution
ax3 = axes[1, 0]
exam_type_counts = df_slim['exam_type_label'].value_counts().sort_index()
colors_combined = ['#ced4da', '#51cf66', '#4c6ef5', '#ffd43b']
bars = ax3.bar(range(len(exam_type_counts)), exam_type_counts.values, 
               color=colors_combined, edgecolor='black', alpha=0.7, linewidth=2)
ax3.set_xticks(range(len(exam_type_counts)))
ax3.set_xticklabels(exam_type_counts.index, rotation=45, ha='right')
ax3.set_ylabel('Count', fontsize=12, fontweight='bold')
ax3.set_title('Exam Type Categories (Combined)', fontsize=13, fontweight='bold')
ax3.grid(alpha=0.3, axis='y')

# Add count and percentage labels
for i, (idx, v) in enumerate(exam_type_counts.items()):
    pct = (v / len(df_slim)) * 100
    ax3.text(i, v + 10, f'{v}\n({pct:.1f}%)', ha='center', fontsize=10, fontweight='bold')

# Plot 4: Summary statistics table
ax4 = axes[1, 1]
ax4.axis('off')

summary_text = f"""
EXAM FEATURES SUMMARY

Total Patients: {len(df_slim):,}

n_imaging_tests (count):
  • Mean: {df_slim['n_imaging_tests'].mean():.2f}
  • Median: {df_slim['n_imaging_tests'].median():.0f}
  • Min-Max: {df_slim['n_imaging_tests'].min():.0f} - {df_slim['n_imaging_tests'].max():.0f}
  • Patients with NO exams: {(df_slim['n_imaging_tests'] == 0).sum()} ({(df_slim['n_imaging_tests'] == 0).mean()*100:.1f}%)
  • Patients with ALL 6 exams: {(df_slim['n_imaging_tests'] == 6).sum()} ({(df_slim['n_imaging_tests'] == 6).mean()*100:.1f}%)

has_radiology_exam:
  • Yes: {(df_slim['has_radiology_exam'] == 1).sum()} ({(df_slim['has_radiology_exam'].mean()*100):.1f}%)
  • No: {(df_slim['has_radiology_exam'] == 0).sum()} ({((1-df_slim['has_radiology_exam'].mean())*100):.1f}%)

has_cardiac_exam:
  • Yes: {(df_slim['has_cardiac_exam'] == 1).sum()} ({(df_slim['has_cardiac_exam'].mean()*100):.1f}%)
  • No: {(df_slim['has_cardiac_exam'] == 0).sum()} ({((1-df_slim['has_cardiac_exam'].mean())*100):.1f}%)

exam_type (combined):
  • {exam_type_map[0]}: {(df_slim['exam_type'] == 0).sum()} ({(df_slim['exam_type'] == 0).mean()*100:.1f}%)
  • {exam_type_map[1]}: {(df_slim['exam_type'] == 1).sum()} ({(df_slim['exam_type'] == 1).mean()*100:.1f}%)
  • {exam_type_map[2]}: {(df_slim['exam_type'] == 2).sum()} ({(df_slim['exam_type'] == 2).mean()*100:.1f}%)
  • {exam_type_map[3]}: {(df_slim['exam_type'] == 3).sum()} ({(df_slim['exam_type'] == 3).mean()*100:.1f}%)
"""

ax4.text(0.1, 0.95, summary_text, transform=ax4.transAxes, fontsize=10,
         verticalalignment='top', family='monospace',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.savefig(r'../plots/1.1_df1_exam_features_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. INDIVIDUAL EXAM FLAGS
print("\n" + "="*80)
print("FEATURE 3: Individual Exam Flags (0/1)")
print("="*80)

for col in exam_cols:
    df_slim[f'has_{col.lower()}'] = df_slim[col].notna().astype(int)
    count = (df_slim[f'has_{col.lower()}'] == 1).sum()
    pct = (count / len(df_slim)) * 100
    print(f"  has_{col.lower():12s}: {count:5d} ({pct:5.1f}%)")

# 4. CROSS-TABULATION: Exam types by gender and age group
print("\n" + "="*80)
print("EXAM PATTERNS BY DEMOGRAPHIC")
print("="*80)

# Create age groups
df_slim['age_group'] = pd.cut(df_slim['age'], bins=[0, 30, 50, 70, 120], 
                               labels=['<30', '30-50', '50-70', '70+'])

print("\nExam Type Distribution by Gender:")
print(pd.crosstab(df_slim['gender'], df_slim['exam_type_label'], margins=True))

print("\nExam Type Distribution by Age Group:")
print(pd.crosstab(df_slim['age_group'], df_slim['exam_type_label'], margins=True))

# Chi-square test
from scipy.stats import chi2_contingency

chi2_gender, p_gender, dof_gender, expected_gender = chi2_contingency(
    pd.crosstab(df_slim['gender'], df_slim['exam_type'])
)
print(f"\nChi-square test (Gender × Exam Type): χ² = {chi2_gender:.4f}, p = {p_gender:.4e}")

chi2_age, p_age, dof_age, expected_age = chi2_contingency(
    pd.crosstab(df_slim['age_group'], df_slim['exam_type'])
)
print(f"Chi-square test (Age Group × Exam Type): χ² = {chi2_age:.4f}, p = {p_age:.4e}")

# Visualize cross-tabulation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# By gender
ax1 = axes[0]
crosstab_gender = pd.crosstab(df_slim['gender'], df_slim['exam_type_label'], normalize='index') * 100
crosstab_gender.plot(kind='bar', ax=ax1, color=['#ced4da', '#51cf66', '#4c6ef5', '#ffd43b'], 
                     edgecolor='black', alpha=0.8, linewidth=1.5)
ax1.set_xlabel('Gender', fontsize=12, fontweight='bold')
ax1.set_ylabel('Percentage (%)', fontsize=12, fontweight='bold')
ax1.set_title('Exam Type Distribution by Gender (χ² p-value: {:.4e})'.format(p_gender), 
              fontsize=13, fontweight='bold')
ax1.legend(title='Exam Type', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
ax1.grid(alpha=0.3, axis='y')
plt.setp(ax1.get_xticklabels(), rotation=0)

# By age group
ax2 = axes[1]
crosstab_age = pd.crosstab(df_slim['age_group'], df_slim['exam_type_label'], normalize='index') * 100
crosstab_age.plot(kind='bar', ax=ax2, color=['#ced4da', '#51cf66', '#4c6ef5', '#ffd43b'], 
                  edgecolor='black', alpha=0.8, linewidth=1.5)
ax2.set_xlabel('Age Group', fontsize=12, fontweight='bold')
ax2.set_ylabel('Percentage (%)', fontsize=12, fontweight='bold')
ax2.set_title('Exam Type Distribution by Age Group (χ² p-value: {:.4e})'.format(p_age), 
              fontsize=13, fontweight='bold')
ax2.legend(title='Exam Type', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
ax2.grid(alpha=0.3, axis='y')
plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig(r'../plots/1.1_df1_exam_patterns_demographic.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("NEW FEATURES CREATED")
print("="*80)
print("\nColumns added to df_slim:")
new_cols = ['n_imaging_tests', 'has_radiology_exam', 'has_cardiac_exam', 
            'exam_type', 'exam_type_label', 'age_group',
            'has_x-ray', 'has_ct', 'has_ultrasound', 'has_cath', 'has_ecg', 'has_mri']
new_cols = [col for col in new_cols if col in df_slim.columns]
for col in new_cols:
    print(f"  ✓ {col}: {df_slim[col].dtype}")

print(f"\ndf_slim shape: {df_slim.shape}")
print(f"df_slim.info():")
print(df_slim[['n_imaging_tests', 'has_radiology_exam', 'has_cardiac_exam', 'exam_type_label', 'age_group']].info())

### Corr

In [ ]:
print("="*80)
print("CORRELATION MATRIX ANALYSIS")
print("="*80)

# Select numeric columns only
numeric_cols = df_slim.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNumeric columns found: {numeric_cols}")
print(f"Total numeric columns: {len(numeric_cols)}\n")

if len(numeric_cols) > 1:
    # Calculate correlation matrix
    correlation_matrix = df_slim[numeric_cols].corr()
    
    print("="*80)
    print("CORRELATION MATRIX")
    print("="*80)
    print(correlation_matrix.round(3))
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # Plot 1: Full correlation heatmap
    ax1 = axes[0]
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.2f', 
                cmap='coolwarm', 
                center=0,
                square=True,
                linewidths=0.5,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=ax1)
    ax1.set_title('Full Correlation Matrix Heatmap', fontsize=14, fontweight='bold')
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    plt.setp(ax1.get_yticklabels(), rotation=0)
    
    # Plot 2: Mask for upper triangle (cleaner view)
    ax2 = axes[1]
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.2f', 
                cmap='coolwarm', 
                center=0,
                square=True,
                linewidths=0.5,
                mask=mask,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=ax2)
    ax2.set_title('Lower Triangle Correlation Matrix (Unique Pairs)', fontsize=14, fontweight='bold')
    plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
    plt.setp(ax2.get_yticklabels(), rotation=0)
    
    plt.tight_layout()
    plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Extract strong correlations (> 0.5 or < -0.5, excluding diagonal)
    print("\n" + "="*80)
    print("STRONG CORRELATIONS (|r| > 0.5, excluding self-correlations)")
    print("="*80)
    
    strong_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:
                strong_corr.append({
                    'Variable 1': correlation_matrix.columns[i],
                    'Variable 2': correlation_matrix.columns[j],
                    'Correlation': corr_value,
                    'Strength': 'Strong Positive' if corr_value > 0 else 'Strong Negative'
                })
    
    if strong_corr:
        strong_corr_df = pd.DataFrame(strong_corr).sort_values('Correlation', key=abs, ascending=False)
        print(f"\nFound {len(strong_corr)} strong correlations:\n")
        print(strong_corr_df.to_string(index=False))
    else:
        print("\nNo correlations with |r| > 0.5 found")
    
    # Moderate correlations (0.3 to 0.5)
    print("\n" + "="*80)
    print("MODERATE CORRELATIONS (0.3 < |r| ≤ 0.5)")
    print("="*80)
    
    moderate_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if 0.3 < abs(corr_value) <= 0.5:
                moderate_corr.append({
                    'Variable 1': correlation_matrix.columns[i],
                    'Variable 2': correlation_matrix.columns[j],
                    'Correlation': corr_value,
                    'Strength': 'Moderate Positive' if corr_value > 0 else 'Moderate Negative'
                })
    
    if moderate_corr:
        moderate_corr_df = pd.DataFrame(moderate_corr).sort_values('Correlation', key=abs, ascending=False)
        print(f"\nFound {len(moderate_corr)} moderate correlations:\n")
        print(moderate_corr_df.to_string(index=False))
    else:
        print("\nNo moderate correlations found (0.3 < |r| ≤ 0.5)")
    
    # Summary statistics
    print("\n" + "="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    
    # Get correlation values (excluding diagonal)
    corr_values = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_values.append(correlation_matrix.iloc[i, j])
    
    corr_values = np.array(corr_values)
    print(f"\nMean correlation: {corr_values.mean():.3f}")
    print(f"Median correlation: {np.median(corr_values):.3f}")
    print(f"Std Dev: {corr_values.std():.3f}")
    print(f"Min: {corr_values.min():.3f}")
    print(f"Max: {corr_values.max():.3f}")
    
    # Distribution of correlations
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.hist(corr_values, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    ax.axvline(corr_values.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {corr_values.mean():.3f}')
    ax.axvline(np.median(corr_values), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(corr_values):.3f}')
    ax.set_xlabel('Correlation Coefficient', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax.set_title('Distribution of Correlation Coefficients', fontsize=13, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig('correlation_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("⚠️ Not enough numeric columns for correlation analysis")
    print(f"Found only {len(numeric_cols)} numeric column(s)")

### Dropunused

In [ ]:
df_slim.info()

In [ ]:
df_slim.columns, len(df_slim.columns)

In [ ]:
# DROP: 'note_id', 'note_type', 'note_seq', 'subject_id_dx', 'storetime', 'HPI', 'physical_exam', 'chief_complaint', 'invasions', 'X-ray', 'CT', 'Ultrasound', 'CATH', 'MRI', 'reports', 'subject_id_dx',  'anchor_year', 'dod', 'has_x-ray', 'has_ct', 'has_ultrasound', 'has_cath', 'has_ecg', 'has_mri', 'age_group'

# keep 'subject_id', 'hadm_id' , icd_code, 'ECG' (for later task 3), charttime
df_slim.drop(columns=[ 'note_id', 'note_type', 'note_seq', 'subject_id_dx', 'storetime', 'HPI', 'physical_exam', 'chief_complaint', 'invasions', 'X-ray', 'CT', 'Ultrasound', 'CATH', 'MRI', 'reports', 'subject_id_dx',  'anchor_year', 'dod', 'has_x-ray', 'has_ct', 'has_ultrasound', 'has_cath', 'has_ecg', 'has_mri', 'age_group'], inplace=True)
df_slim.columns, len(df_slim.columns)

In [ ]:
df_slim[['exam_type', 'exam_type_label']].tail()

In [ ]:
df_slim['has_cardiac_exam'].value_counts(), df_slim['has_radiology_exam'].value_counts()

In [ ]:
df_slim['exam_type_label'].value_counts()

In [ ]:
df_slim['is_dead'].value_counts()

In [ ]:
df_slim.drop(columns=['long_title', 'has_radiology_exam' ], inplace=True)
df_slim.columns, len(df_slim.columns)


## Save SLim

In [ ]:
df_slim.to_csv(f"{DATA_DIR}/heart_diagnoses_1_cleaned_slim.csv", index=False)